### Date: April 19, 2025

![NVIDIA](images/nvidia.png)

# Chain-of-Thought Prompting

In this notebook, you will learn about one of the most famous prompting techniques called Chain-of-Thought prompting.

---

## Imports

In [1]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

---

## Create a Model Instance

In [2]:
base_url = 'http://llama:8000/v1'
model = 'meta/llama-3.1-8b-instruct'
llm = ChatNVIDIA(base_url=base_url, model=model, temperature=0)

---

## LLMs Jump to Conclusions

Since LLMs are primarily designed to generate what is most likely to come next in a stream of text, it's no surprise that they can often "jump to conclusions" in scenarios that actually require complex reasoning.

Consider the following example, which challenges the LLM do do some multiplication. Before invoking the LLM, we'll use Python to provide us with the actual answer.

In [3]:
345*888

306360

In [4]:
print(llm.invoke('What is 345 * 888?').content)

The result of multiplying 345 by 888 is:

306,720


This answer is incorrect, but given what we know about LLMs, again, that they are designed primarily to generate the most likely response given the data they were trained on, it may not come as a surprise.

Significant changes are likely to occur in coming years, but as it stands, and in spite of the common use of terms like "Artificial **Intelligence**", LLMs are not well suited, at least not without additional user support, to deliberate step by step reasoning. Rather, LLMs are, because of their design, well suited to jumping to conclusions.

---

## Hallucination

This phenomenon of LLMs generating incorrect information, often with what appears like "confidence" to outside observers, is called **hallucination**. Because of their design, an LLM will usually say something incorrect (unbeknownst to the LLM itself) rather than say nothing at all. While it's true that LLMs over time are hallucinating less, and that over time they will continue to improve in this regard, hallucination is an inherent characteristic of LLM generation.

Regarding LLM hallucination, you need to commit the following golden rules to memory:
1. All LLMs hallucinate.
2. You are responsible for what your LLMs generate in your applications.

---

## Chain-of-Thought Prompting

Returning to the main subject matter of this notebook, while LLMs are not well-suited to long-form deliberate reasoning, there are techniques we can utilize to guide LLMs towards taking a more deliberate step by step approach when the task demands it. One such technique is call Chain-of-Thought

Introduced in 2023, [Chain-of-Thought(CoT) prompting](https://arxiv.org/pdf/2201.11903) is still one of the most popular prompting techniques. It supports complex reasoning capabilities by encouraging LLMs to break down complex problems into intermediate steps. CoT prompting involves providing examples that demonstrate step-by-step reasoning to solve a problem.

Here is the example from the paper that introduced CoT prompting.

![Chain-of-Thought](images/Chain.png)

As you can see in the right-hand-side example, by modeling for the LLM that it ought to break the complex task into smaller subtasks that it is more likely to be able to perform, the model is given "time to think" and can do better at tasks requiring complex reasoning.

---

## Chain-of-Though Multiplication

Let's try a CoT prompt for 3-digit long multiplication. We will start with an example multiplication prompt.

In [5]:
example_problem = 'What is 678 * 789?'

Next, we'll supply a chain-of-thought example of working through the problem step by step.

In [6]:
example_cot = '''\
Let me break this down into steps. First I'll break down 789 into hundreds, tens, and ones:

789 -> 700 + 80 + 9

Next I'll multiply 678 by each of these values, storing the intermediate results:

678 * 700 -> 678 * 7 * 100 -> 4746 * 100 -> 474600

My first intermediate result is 474600.

678 * 80 -> 678 * 8 * 10 -> 5424 * 10 -> 54240

My second intermediate result is 54240.

678 * 9 -> 6102

My third intermediate result is 6102.

My three intermediate results are 474600, 54240, and 6102.

Adding the first two intermediate results I get 474600 + 54240 -> 528840.

Adding 528840 to the last intermediate result I get 528840 + 6102 -> 534942

The final result is 534942.
'''

With our example problem and example CoT response, we can construct a one-shot prompt template.

In [7]:
multiplication_template = ChatPromptTemplate.from_messages([
    ('human', example_problem),
    ('ai', example_cot),
    ('human', '{long_multiplication_prompt}')
])

We'll now use this template in a simple chain.

In [8]:
multiplication_chain = multiplication_template | llm | StrOutputParser()

Let's revisit the original multiplication problem we tasked the LLM with to see if it does any better.

In [9]:
print(multiplication_chain.invoke('What is 345 * 888?'))

Let's break this down into steps. First I'll break down 888 into hundreds, tens, and ones:

888 -> 800 + 80 + 8

Next I'll multiply 345 by each of these values, storing the intermediate results:

345 * 800 -> 345 * 8 * 100 -> 2760 * 100 -> 276000

My first intermediate result is 276000.

345 * 80 -> 345 * 8 * 10 -> 2760 * 10 -> 27600

My second intermediate result is 27600.

345 * 8 -> 2760

My third intermediate result is 2760.

My three intermediate results are 276000, 27600, and 2760.

Adding the first two intermediate results I get 276000 + 27600 -> 303600.

Adding 303600 to the last intermediate result I get 303600 + 2760 -> 306360

The final result is 306360.


In [10]:
345*888

306360

As you can see, the LLM followed the example in our CoT prompt, taking the problem step by step, and in this case, generated the correct response.

---

## Zero-shot Chain-of-Thought Prompting

CoT prompting can get really verbose, and as we mentioned earlier, you shouldn't be shy about writing long prompts when needed to accomplish your goals.

That said, there are more elegant ways to leverage CoT methods with LLMs. One such variant is called [Zero-shot CoT](https://arxiv.org/abs/2205.11916). This prompting technique simply adds "Let's think step by step" to the prompt without supplying verbose CoT examples, and it has been shown to work quite well.

Let's construct a new prompt template to try Zero-shot CoT on long multiplication, following the example in the paper, simply supplying the language "Let's think step by step".

In [11]:
zero_shot_cot_prompt = ChatPromptTemplate([
    ("human", "{long_multiplication_prompt} Let's think step by step.")
])

In [12]:
zero_shot_multiplication_chain = zero_shot_cot_prompt | llm | StrOutputParser()

In [13]:
print(zero_shot_multiplication_chain.invoke('What is 345 * 888?'))

To calculate 345 * 888, let's break it down step by step.

First, we can multiply 300 by 888:
300 * 888 = 266,400

Next, we can multiply 40 by 888:
40 * 888 = 35,520

Then, we can multiply 5 by 888:
5 * 888 = 4,440

Now, we add up the results of each multiplication:
266,400 + 35,520 = 301,920
301,920 + 4,440 = 306,360

So, 345 * 888 = 306,360.


In [14]:
345*888

306360

For our 3-digit long multiplication problem, simply prompting the model to think step by step performed as well as our more verbose CoT prompt.

---

## CoT Prompting In Practice

As with many other aspects of prompt engineering, especially given the variety of LLMs available (both now and in the future) it's difficult to give hard and fast rules about when and how exactly to employ CoT prompting. That said, we can offer you some good general guidelines to follow.

- Develop your prompts iteratively. Start simple, try zero-shot CoT when you think it might be helpful, and expand to use more verbose example-based CoT prompting when the need arises.
- Consider the use of external, non-LLM tools for tasks that LLMs may not be well-suited for (like math). We are going to look at tool use later in the workshop but the general idea is that LLMs are incredible, but not necessarily for every single task. Reflect if you will on the case of long multiplication about how fast, reliable, and effective simple Python is. Don't fall into the old adage _"If all you have is a hammer, everything looks like a nail."_ LLMs are only one of your many tools to use while building LLM-based applications.

---

## Exercise: Use an LLM to Solve a Word Problem

For this exercise, use what you've learned in this notebook to a correct response back from our LLM to the following word problem.

In [15]:
word_problem = """Michael's car travels at 40 miles per hour. He is driving from 1 PM to 4 PM and then \
travels back at a rate of 25 miles per hour due to heavy traffic. How long in \
terms of minutes did it take him to get back?"""

Before beginning your work, here is the actual solution to the word problem.

Michael drove 40 miles per hour for 3 hours, which means he drove 120 miles.

To come back at a rate of 25 miles per hour, it would have taken him 4.8 hours (120 miles / 25 mph) which is equivalent to **288** minutes (4.8 hours * 60 minutes/hour).

The correct answer is **288**.

Feel free to check out the *Solution* below if you get stuck.

### Your Work Here

### Solution

There are lots of ways to solve this, but we opted to include a system message encouring the LLM to always breaking their work into smaller tasks and showing their work, and then, appending the cannonical zero-shot CoT prompt to whatever the human user passes in.

In [16]:
template = ChatPromptTemplate.from_messages([
    ('system', 'You are an expert word problem solver. You always break your problem down into smaller tasks and show your work.'),
    ('human', '{prompt}\n\nLet\'s think step by step.')
])

In [17]:
chain = template | llm | StrOutputParser()

In [18]:
print(chain.invoke(word_problem))

Let's break this problem down into smaller tasks.

**Task 1: Find the total time Michael spent driving from 1 PM to 4 PM**

Since Michael drove from 1 PM to 4 PM, we need to find the time difference between 4 PM and 1 PM.

4 PM - 1 PM = 3 hours

**Task 2: Convert the time from hours to minutes**

Since there are 60 minutes in an hour, we multiply the number of hours by 60 to get the total minutes.

3 hours × 60 minutes/hour = 180 minutes

**Task 3: Find the distance Michael traveled from 1 PM to 4 PM**

We know Michael's car travels at 40 miles per hour. To find the distance, we multiply the speed by the time.

Distance = Speed × Time
= 40 miles/hour × 3 hours
= 120 miles

**Task 4: Find the time it took Michael to travel back**

Since Michael traveled 120 miles at a speed of 25 miles per hour, we can use the formula:

Time = Distance ÷ Speed
= 120 miles ÷ 25 miles/hour
= 4.8 hours

**Task 5: Convert the time from hours to minutes**

We multiply the number of hours by 60 to get the tot